In [337]:
# import nltk
# import streamlit as st
# import pandas as pd
# import spacy
# import pyinflect
# import random     
# import numpy as np



#from create_exercise import Create_exercise

# nlp = spacy.load("en_core_web_sm") 

In [338]:
import nltk
import streamlit as st
import pandas as pd
import spacy
import pyinflect
import random     
import numpy as np

from tqdm import tqdm

nlp = spacy.load("en_core_web_sm") 

# Главный класс приложения
class Create_exercise:
    def __init__(self, task='', answer=''):
        #self.df = df
        #self.options=options
        self.task = task
        self.answer = answer
        self.exercise_type = exercise_type

 # Загрузка текста   
    def get_text(self):
        with open('Little_Red_Cap_ Jacob_and_Wilhelm_Grimm.txt') as f:
            self.text = f.read()
            self.text = self.text.replace('\n','')
            return self.text
# Очистка текста        
    def clear_text(self, text):
        self.text = self.text.lower()
        self.text = self.text.replace('"', '')
        self.text = self.text.replace(',', '')
        self.text = self.text.replace(':', '')
        self.text = self.text.replace('-"', '')
        return self.text

    def create_df(self):
        self.df = pd.DataFrame(columns=['sentence', 'options', 'answer', 'task', 'result'])
        # self.text = text
        # tokens_sens = nltk.tokenize.sent_tokenize(self.text, language='english')
        # df_sentences = pd.DataFrame({'sentence': tokens_sens})
        # df_sentences["sentence"]= df_sentences.apply(lambda x: x['sentence'].replace('.', ''), axis=1)
        return self.df

# Токенизация по предложениям и создание ДФ
    def tokenization(self, text):
        self.text=text
        self.tokens_sens = nltk.tokenize.sent_tokenize(self.text, language='english')
        self.df_sentences = pd.DataFrame({'sentence': self.tokens_sens})
        self.df_sentences["sentence"]= self.df_sentences.apply(lambda x: x['sentence'].replace('.', ''), axis=1)    
        return self.df_sentences

# Обработка упражнения и вывод итогового ДФ
    def select_exercise(self, df_sentences, options, exercise_type):

        print('exercise_type=', exercise_type)
        
        try:
            self.df = pd.read_csv('df_VERB.csv')
            print('Загрузили фрейм')
            display(self.df)
            print(option)
            return self.df
        except:
            print("Сработало исключение")
        print("Код работает дальше")
        for sentence in df_sentences.sentence:
            for token in nlp(str(sentence)):
                if token.pos_=='VERB' and exercise_type == 'Выберите правильную форму глагола':
                    self.answer = [token.text for token in nlp(str(sentence)) if token.pos_=='VERB']
                    options.append(list(set([token._.inflect('VB'), token._.inflect('VBN'), token._.inflect('VBP'), token._.inflect('VBZ'), token._.inflect('VBG'), token._.inflect('VBD')])))
                    self.task = token.pos_
                    write_it_df=1
                elif token.pos_=='ADJ'and exercise_type == 'Выбор правильного прилагательного':
                    self.answer = [token.text for token in nlp(str(sentence)) if token.pos_=='ADJ']
                    options.append([token.text, token._.inflect('JJS')])
                    self.task = token.pos_
                    write_it_df=1
            
                elif exercise_type ==  'Расставьте в правильном порядке слова предложения'  and len(nlp(str(sentence))) < 9:
                    options = [token.text for token in nlp(str(sentence))]
                    options = [options] * len(options)
                    self.answer = [token.text for token in nlp(str(sentence))]
                        
                    write_it_df=1
                    self.task = 'order_words'
                else: pass                

            if exercise_type == 'Выберите правильный артикль':
                self.task = 'articles'
                self.answer=[]
                split_string = sentence.split(" ")
                #st.write(test_string)
                #st.write(len(test_string))
                if len(split_string) in range (3, 20):
                    st.write(len(split_string))
                for i in split_string:
                    for j in ['a', 'the', 'an']:
                        if i==j:
                            self.answer.append(i)
                            options.append([' a ', ' the ', ' an '])
                            break  
                self.answer = list(map(lambda x: ' '+ x + ' ', self.answer))  
                write_it_df=1     

            if len(nlp(str(sentence))) in range(3, 20) and len(self.answer) > 0 and write_it_df==1:                    
                self.df.loc[len(self.df)]=[sentence, options, self.answer, self.task, []]  

            # сбрасываем переменные    
            options=[]  
            write_it_df=0    
            self.answer=[]        
        #print("Упражнение:",exercise_type)
        self.df["sentence_hidden"] = self.df["sentence"]
        for index, row in self.df.iterrows(): 
            for i in row.answer:
                #print("Упражнение:",exercise_type)
                if exercise_type == 'Расставьте в правильном порядке слова предложения':
                    self.df["sentence_hidden"][index] = '_________________'
                else: self.df["sentence_hidden"][index] = self.df["sentence_hidden"][index].replace(i, ' ___ ')

        filename = f'df_{self.task}.csv'
        self.df.to_csv(filename, index=False) 
        return self.df

In [339]:
df

,sentence,options,answer,task,result,sentence_hidden
0,once upon a time there was a sweet little girl,"[['is', 'be', 'been', 'am', 'was', 'being']]",['was'],VERB,[],once upon a time there ___ a sweet little girl
1,once she gave her a little cap made of red velvet,"[['giving', 'gives', 'gave', 'given', 'give'],...","['gave', 'made']",VERB,[],once she ___ her a little cap ___ of red v...
2,take them to your grandmother,"[['taking', 'take', 'taken', 'takes', 'took']]",['take'],VERB,[],___ them to your grandmother
3,she is sick and weak and they will do her well,"[['did', 'does', 'doing', 'do', 'done']]",['do'],VERB,[],she is sick and weak and they will ___ her well
4,mind your manners and give her my greetings,"[['minding', 'minded', 'mind', 'minds'], ['giv...","['mind', 'give']",VERB,[],___ your manners and ___ her my greetings


In [340]:
nltk.download('punkt')

random.seed(42)

[nltk_data] Downloading package punkt to /Users/Sergey/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [341]:
exercise_type = 'Выберите правильную форму глагола'   
create_exercise = Create_exercise()   

text = st.text_area('Текст', create_exercise.get_text())

#text = create_exercise.contracted_text(text)

#exercise_type = st.sidebar.selectbox('Выберите тип упражнения:', ['', 'Выберите правильную форму глагола',  'Выбор правильного прилагательного', 'Выберите правильный артикль', 'Расставьте в правильном порядке слова предложения'], format_func=lambda x: 'Ничего не выбрано' if x == '' else x)
  

In [342]:
df

,sentence,options,answer,task,result,sentence_hidden
0,once upon a time there was a sweet little girl,"[['is', 'be', 'been', 'am', 'was', 'being']]",['was'],VERB,[],once upon a time there ___ a sweet little girl
1,once she gave her a little cap made of red velvet,"[['giving', 'gives', 'gave', 'given', 'give'],...","['gave', 'made']",VERB,[],once she ___ her a little cap ___ of red v...
2,take them to your grandmother,"[['taking', 'take', 'taken', 'takes', 'took']]",['take'],VERB,[],___ them to your grandmother
3,she is sick and weak and they will do her well,"[['did', 'does', 'doing', 'do', 'done']]",['do'],VERB,[],she is sick and weak and they will ___ her well
4,mind your manners and give her my greetings,"[['minding', 'minded', 'mind', 'minds'], ['giv...","['mind', 'give']",VERB,[],___ your manners and ___ her my greetings


In [343]:
num_of_sentenses = 5

text = create_exercise.clear_text(text)

In [344]:
options = []

df = create_exercise.create_df()

df_sentences = create_exercise.tokenization(text)    
df = create_exercise.select_exercise(df_sentences, options, exercise_type)

key=0
num_of_sentenses = 5
df = df[0:num_of_sentenses]

exercise_type= Выберите правильную форму глагола
Загрузили фрейм


,sentence,options,answer,task,result,sentence_hidden
0,once upon a time there was a sweet little girl,"[['is', 'be', 'been', 'am', 'was', 'being']]",['was'],VERB,[],once upon a time there ___ a sweet little girl
1,once she gave her a little cap made of red velvet,"[['giving', 'gives', 'gave', 'given', 'give'],...","['gave', 'made']",VERB,[],once she ___ her a little cap ___ of red v...
2,take them to your grandmother,"[['taking', 'take', 'taken', 'takes', 'took']]",['take'],VERB,[],___ them to your grandmother
3,she is sick and weak and they will do her well,"[['did', 'does', 'doing', 'do', 'done']]",['do'],VERB,[],she is sick and weak and they will ___ her well
4,mind your manners and give her my greetings,"[['minding', 'minded', 'mind', 'minds'], ['giv...","['mind', 'give']",VERB,[],___ your manners and ___ her my greetings
...,...,...,...,...,...,...
36,carry the water that i boiled them with to the...,"[['carry', 'carries', 'carrying', 'carried'], ...","['carry', 'boiled']",VERB,[],___ the water that i ___ them with to the ...
37,little red cap carried water until the large l...,"[['carry', 'carries', 'carrying', 'carried']]",['carried'],VERB,[],little red cap ___ water until the large lar...
38,the smell of sausage arose into the wolf's nose,"[['arisen', 'arises', 'arose', 'arising', 'ari...",['arose'],VERB,[],the smell of sausage ___ into the wolf's nose
39,he slid off the roof fell into the trough and ...,"[['slide', 'slid', 'slides', 'sliding'], ['fal...","['slid', 'fell', 'drowned']",VERB,[],he ___ off the roof ___ into the trough an...


[


In [345]:
#options
df

,sentence,options,answer,task,result,sentence_hidden
0,once upon a time there was a sweet little girl,"[['is', 'be', 'been', 'am', 'was', 'being']]",['was'],VERB,[],once upon a time there ___ a sweet little girl
1,once she gave her a little cap made of red velvet,"[['giving', 'gives', 'gave', 'given', 'give'],...","['gave', 'made']",VERB,[],once she ___ her a little cap ___ of red v...
2,take them to your grandmother,"[['taking', 'take', 'taken', 'takes', 'took']]",['take'],VERB,[],___ them to your grandmother
3,she is sick and weak and they will do her well,"[['did', 'does', 'doing', 'do', 'done']]",['do'],VERB,[],she is sick and weak and they will ___ her well
4,mind your manners and give her my greetings,"[['minding', 'minded', 'mind', 'minds'], ['giv...","['mind', 'give']",VERB,[],___ your manners and ___ her my greetings


In [346]:
exercise_type = 'Расставьте в правильном порядке слова предложения'   
exercise_type = 'Расставьте в правильном порядке слова предложения'   
exercise_type = 'Расставьте в правильном порядке слова предложения'   
#exercise_type = 'Выберите правильную форму глагола'   

In [347]:
pd.read_csv('df_VERB_old.csv').head(5)

,sentence,options,answer,task,result,sentence_hidden
0,once upon a time there was a sweet little girl,"[['is', 'be', 'been', 'am', 'was', 'being']]",['was'],VERB,[],once upon a time there ___ a sweet little girl
1,once she gave her a little cap made of red velvet,"[['giving', 'gives', 'gave', 'given', 'give'],...","['gave', 'made']",VERB,[],once she ___ her a little cap ___ of red v...
2,take them to your grandmother,"[['taking', 'take', 'taken', 'takes', 'took']]",['take'],VERB,[],___ them to your grandmother
3,she is sick and weak and they will do her well,"[['did', 'does', 'doing', 'do', 'done']]",['do'],VERB,[],she is sick and weak and they will ___ her well
4,mind your manners and give her my greetings,"[['minding', 'minded', 'mind', 'minds'], ['giv...","['mind', 'give']",VERB,[],___ your manners and ___ her my greetings


In [348]:
pd.read_csv('df_VERB_new.csv').head(5)

,sentence,options,answer,task,result,sentence_hidden
0,once upon a time there was a sweet little girl,"[['is', 'be', 'been', 'am', 'was', 'being']]",['was'],VERB,[],once upon a time there ___ a sweet little girl
1,once she gave her a little cap made of red velvet,"[['giving', 'gives', 'gave', 'given', 'give'],...","['gave', 'made']",VERB,[],once she ___ her a little cap ___ of red v...
2,take them to your grandmother,"[['taking', 'take', 'taken', 'takes', 'took']]",['take'],VERB,[],___ them to your grandmother
3,she is sick and weak and they will do her well,"[['did', 'does', 'doing', 'do', 'done']]",['do'],VERB,[],she is sick and weak and they will ___ her well
4,mind your manners and give her my greetings,"[['minding', 'minded', 'mind', 'minds'], ['giv...","['mind', 'give']",VERB,[],___ your manners and ___ her my greetings


In [349]:


pd.set_option('display.max_rows', 20) 
print(row['options'][0])
for index, row in df.iterrows():
    print(str(row['sentence_hidden']))
    print(str(row['sentence']))
    
    option = []
    for i in range(len(row['options'])):
        key+=1
        option = row['options'][i]
        random.shuffle(option)
        option = ['–––'] + option
        #df['result'][index] =  st.selectbox('nolabel', option, label_visibility="hidden", key =str(key) )
        df['result'][index] =  'any'

        if df['result'][index] == '–––':
            pass

        elif df['result'][index] == str(row['answer'][i]):
            print('Это правльный ответ')
        
        else:
            print('Попробуйте еще раз')
    print('----------------------')       
print(exercise_type)         

[
once upon a time there  ___  a sweet little girl
once upon a time there was a sweet little girl


TypeError: can only concatenate list (not "str") to list